# Процесс моделирования

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error


%matplotlib inline

## Определение бизнес-требований

На этом этапе мы определим, что именно нужно сделать и зачем это нужно сделать. 

Такая плстановка вопроса помогает понять, ножно ли нам вообще применять методы МО в данном конкретном случае.

После этого мы запрашиваем необходимые данные и начинаем работу. При необходимости мы сможем вернуться на этот шаг.

## Подготовка данных

Загрузим данные

In [2]:
dataset = load_boston()

In [3]:
dataset.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

In [4]:
X = pd.DataFrame(dataset['data'], columns=dataset['feature_names'])
y = pd.Series(dataset['target'])

In [5]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [6]:
print(dataset['DESCR'][:1220])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

Проверим наличие пропусков

In [7]:
X.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
dtype: int64

In [8]:
y.head()

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
dtype: float64

Посмотрим на корреляцию признаков с целевой переменной:

In [9]:
trimmed_feature_names = []

for i in range(X.shape[1]):
    correlation = np.corrcoef(X.iloc[:, i], y)[0, 1]
    if abs(correlation) > 0.5:
        feature_name = dataset['feature_names'][i]
        print(feature_name, correlation)
        trimmed_feature_names.append(feature_name)

RM 0.695359947071539
PTRATIO -0.5077866855375615
LSTAT -0.7376627261740148


In [10]:
X_trimmed = X[trimmed_feature_names]
X_trimmed.head()

# Использование предобработки, в данном случае нужно эксперементировать еще, так как эта предобработка ухудшает результат
# X = X_trimmed

,RM,PTRATIO,LSTAT
0,6.575,15.3,4.98
1,6.421,17.8,9.14
2,7.185,17.8,4.03
3,6.998,18.7,2.94
4,7.147,18.7,5.33


## Разбиение выборки

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', y_train.shape)
print('y_test:', y_test.shape)

X_train: (404, 13)
X_test: (102, 13)
y_train: (404,)
y_test: (102,)


## Обучение модели

### Линейная регрессия

In [12]:
linear_predictor = LinearRegression()
linear_predictor.fit(X_train, y_train)
linear_y_pred = linear_predictor.predict(X_test)

### Градиентный бустинг

In [13]:
gradient_predictor = GradientBoostingRegressor(n_estimators=5000, learning_rate=0.01)
gradient_predictor.fit(X_train, y_train)
gradient_y_pred = gradient_predictor.predict(X_test)

## Валидация модели

In [14]:
linear_result = mean_absolute_error(y_test, linear_y_pred)
gradient_result = mean_absolute_error(y_test, gradient_y_pred)

print('Linear:', linear_result)
print('Gradient:', gradient_result)

Linear: 3.296334739383816
Gradient: 1.9829196155836888


***
**!** Чем меньше полученное число, тем лучше!
***